In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)
import tensorflow.keras.layers as layers
from tensorflow.keras.applications import vgg16, vgg19, densenet, resnet_v2, inception_v3, resnet50, resnet
from tqdm import tqdm
import numpy as np
import gc
from pathlib import Path
import sys
try:
    from utils.data import dataset_definition
except ModuleNotFoundError:
    sys.path.insert(0, str(Path('.').resolve().parent))
    from utils.data import dataset_definition

In [3]:
def make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model='vgg16', data_augmentation=False):
    if transfer_learning:
        weights = 'imagenet'
    else:
        weights = None
    
    if base_model == 'vgg16':
        base_model_net = vgg16.VGG16(include_top=False, weights=weights)
        preprocess_layer = vgg16.preprocess_input
    if base_model == 'vgg19':
        base_model_net = vgg19.VGG19(include_top=False, weights=weights)
        preprocess_layer = vgg19.preprocess_input
    elif base_model == 'densenet201':
        base_model_net = densenet.DenseNet201(include_top=False, weights=weights)
        preprocess_layer = densenet.preprocess_input
    elif base_model == 'densenet169':
        base_model_net = densenet.DenseNet169(include_top=False, weights=weights)
        preprocess_layer = densenet.preprocess_input
    elif base_model == 'densenet121':
        base_model_net = densenet.DenseNet121(include_top=False, weights=weights)
        preprocess_layer = densenet.preprocess_input
    elif base_model == 'resnet152v2':
        base_model_net = resnet_v2.ResNet152V2(include_top=False, weights=weights)
        preprocess_layer = resnet_v2.preprocess_input
    elif base_model == 'resnet50':
        base_model_net = resnet50.ResNet50(include_top=False, weights=weights)
        preprocess_layer = resnet50.preprocess_input
    elif base_model == 'resnet152':
        base_model_net = resnet.ResNet152(include_top=False, weights=weights)
        preprocess_layer = resnet.preprocess_input 
    elif base_model == 'resnet101':
        base_model_net = resnet.ResNet101(include_top=False, weights=weights)
        preprocess_layer = resnet.preprocess_input
    elif base_model == 'inception_v3':
        base_model_net = inception_v3.InceptionV3(include_top=False, weights=weights)
        preprocess_layer = inception_v3.preprocess_input
    
    data_augmentation = tf.keras.Sequential([
        layers.experimental.preprocessing.RandomFlip('horizontal'),
        layers.experimental.preprocessing.RandomRotation(0.2),
    ])
    
    inputs = layers.Input(shape=(img_height, img_width, 3))
    if data_augmentation:
        x = data_augmentation(inputs)
    else:
        x = preprocess_layer(inputs)
    outputs = base_model_net(x, training=False)
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [4]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))): # if value ist tensor
        value = value.numpy() # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a floast_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_array(array):
    array = tf.io.serialize_tensor(array)
    return array

def parse_single_image(image, label):
    #define the dictionary -- the structure -- of our single example
    data = {
        'height' : _int64_feature(image.shape[0]),
        'width' : _int64_feature(image.shape[1]),
        'depth' : _int64_feature(image.shape[2]),
        'raw_image' : _bytes_feature(serialize_array(image)),
        'label' : _int64_feature(label)
    }
    #create an Example, wrapping the single features
    out = tf.train.Example(features=tf.train.Features(feature=data))
    
    return out

def write_images(images, labels, filename:str="images"):
    filename= str(Path(filename+".tfrecords").resolve())
    writer = tf.io.TFRecordWriter(filename)
    count = 0
    
    for index in range(len(images)):
        current_image = images[index]
        current_label = labels[index]
        
        out = parse_single_image(image=current_image, label=current_label)
        writer.write(out.SerializeToString())
        count += 1
    
    writer.close()
    #print(f"Wrote {count} elements to TFRecord")
    return count

def write_tfrecord_dataset(dataset, model, file_path):
    images_concat = None
    labels_concat = None
    batches_count = 0
    batches_count_name = 0
    for images, labels in tqdm(dataset):
        if images_concat is None:
            images_concat = model.predict(images)
        else:
            images_concat = np.concatenate([images_concat, model.predict(images)])

        if labels_concat is None:
            labels_concat = labels.numpy()
        else:
            labels_concat = np.concatenate([labels_concat, labels.numpy()])

        batches_count = batches_count + 1

        if batches_count == n_batches_concat:
            batches_count = 0
            batches_count_name = batches_count_name + 1
            write_images(images_concat, labels_concat, filename=file_path + str(batches_count_name))
            images_concat = None
            labels_concat = None
            gc.collect();

In [5]:
def tensor_argmax(image, label):
    return image, tf.math.argmax(label, axis=1)

# Resnet152 V2

#### Patch Camelyon dataset

In [15]:
file_path = '../data/tfrecords/patch_camelyon/resnet152v2'
n_batches_concat = 10
batch_size = 64
data_augmentation = False

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model='resnet152v2', data_augmentation=data_augmentation)

2022-07-18 14:29:47.927 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-18 14:29:47.955 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-18 14:29:47.955 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-18 14:29:48.006 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-18 14:29:48.008 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-18 14:29:48.008 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [10]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|████████████████████████████████████████████████████████████████████████████| 4096/4096 [1:05:40<00:00,  1.04it/s]


In [11]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [08:42<00:00,  1.02s/it]


In [12]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 512/512 [09:10<00:00,  1.08s/it]


model output size for one image:

In [7]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 7, 7, 2048)


#### Oxford Flowers 102 dataset

In [6]:
file_path = '../data/tfrecords/oxford_flowers102/resnet152v2'
n_batches_concat = 10
batch_size = 64
data_augmentation = False
base_model = 'resnet152v2'

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='oxford_flowers102', batch_size=batch_size, img_height=224, img_width=224)
train_ds = train_ds.map(tensor_argmax)
test_ds = test_ds.map(tensor_argmax)
valid_ds = valid_ds.map(tensor_argmax)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model=base_model, data_augmentation=data_augmentation)

2022-07-18 14:32:08.706 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\oxford_flowers102\2.1.1
2022-07-18 14:32:08.721 INFO    absl: Reusing dataset oxford_flowers102 (C:\Users\lucas\tensorflow_datasets\oxford_flowers102\2.1.1)
2022-07-18 14:32:08.721 INFO    absl: Constructing tf.data.Dataset oxford_flowers102 for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\oxford_flowers102\2.1.1
2022-07-18 14:32:09.210 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\oxford_flowers102\2.1.1
2022-07-18 14:32:09.210 INFO    absl: Reusing dataset oxford_flowers102 (C:\Users\lucas\tensorflow_datasets\oxford_flowers102\2.1.1)
2022-07-18 14:32:09.221 INFO    absl: Constructing tf.data.Dataset oxford_flowers102 for split ['test'], from C:\Users\lucas\tensorflow_datasets\oxford_flowers102\2.1.1


Store features:

In [7]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:23<00:00,  1.49s/it]


In [ ]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

 62%|██████████████████████████████████████████████████▋                               | 60/97 [00:59<00:50,  1.36s/it]

In [ ]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

model output size for one image:

In [ ]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

# VGG16

#### Patch Camelyon dataset

In [6]:
file_path = '../data/tfrecords/patch_camelyon/vgg16'
n_batches_concat = 10
batch_size = 128
data_augmentation = False

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model='vgg16', data_augmentation=data_augmentation)

2022-07-14 13:47:02.529 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-14 13:47:02.541 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-14 13:47:02.542 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-14 13:47:03.479 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-14 13:47:03.481 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-14 13:47:03.481 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [7]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████| 2048/2048 [47:13<00:00,  1.38s/it]


In [8]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [05:59<00:00,  1.40s/it]


In [9]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [06:08<00:00,  1.44s/it]


model output size for one image:

In [7]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 7, 7, 512)


# Inception V3

#### Patch Camelyon dataset

In [6]:
file_path = '../data/tfrecords/patch_camelyon/inceptionv3'
n_batches_concat = 10
batch_size = 128
data_augmentation = False

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model='inception_v3', data_augmentation=data_augmentation)

2022-07-14 23:21:28.687 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-14 23:21:28.718 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-14 23:21:28.718 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-14 23:21:29.286 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-14 23:21:29.288 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-14 23:21:29.289 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [7]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████| 2048/2048 [26:04<00:00,  1.31it/s]


In [8]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [03:14<00:00,  1.32it/s]


In [9]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [03:14<00:00,  1.32it/s]


model output size for one image:

In [10]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 5, 5, 2048)


# VGG19

#### Patch Camelyon dataset

In [6]:
file_path = '../data/tfrecords/patch_camelyon/vgg19'
n_batches_concat = 10
batch_size = 128
data_augmentation = False
base_model = 'vgg19'

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model=base_model, data_augmentation=data_augmentation)

2022-07-15 10:34:49.084 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-15 10:34:49.084 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-15 10:34:49.084 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-15 10:34:49.141 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-15 10:34:49.143 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-15 10:34:49.144 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [7]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████| 2048/2048 [53:12<00:00,  1.56s/it]


In [8]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [06:32<00:00,  1.53s/it]


In [9]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [06:37<00:00,  1.55s/it]


model output size for one image:

In [10]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 7, 7, 512)


# Resnet 50

#### Patch Camelyon dataset

In [5]:
file_path = '../data/tfrecords/patch_camelyon/resnet50'
n_batches_concat = 10
batch_size = 128
data_augmentation = False
base_model = 'resnet50'

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model=base_model, data_augmentation=data_augmentation)

2022-07-15 17:31:44.089 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-15 17:31:44.120 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-15 17:31:44.121 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-15 17:31:44.888 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-15 17:31:44.901 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-15 17:31:44.901 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [6]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████| 2048/2048 [44:19<00:00,  1.30s/it]


In [7]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [05:30<00:00,  1.29s/it]


In [8]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [05:22<00:00,  1.26s/it]


model output size for one image:

In [9]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 7, 7, 2048)


# Densenet 121

#### Patch Camelyon dataset

In [5]:
file_path = '../data/tfrecords/patch_camelyon/densenet121'
n_batches_concat = 10
batch_size = 128
data_augmentation = False
base_model = 'densenet121'

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model=base_model, data_augmentation=data_augmentation)

2022-07-16 15:32:44.455 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-16 15:32:44.474 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-16 15:32:44.475 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-16 15:32:45.018 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-16 15:32:45.025 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-16 15:32:45.025 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [6]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████| 2048/2048 [37:04<00:00,  1.09s/it]


In [7]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [04:49<00:00,  1.13s/it]


In [8]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [04:41<00:00,  1.10s/it]


model output size for one image:

In [6]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 7, 7, 1024)


# Densenet 201

#### Patch Camelyon dataset

In [5]:
file_path = '../data/tfrecords/patch_camelyon/densenet201'
n_batches_concat = 10
batch_size = 128
data_augmentation = False
base_model = 'densenet201'

train_ds, test_ds, valid_ds, class_names = dataset_definition(sample_dataset='patch_camelyon', batch_size=batch_size, img_height=224, img_width=224)
model = make_base_model(img_height=224, img_width=224, transfer_learning=True, base_model=base_model, data_augmentation=data_augmentation)

2022-07-18 13:39:56.705 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-18 13:39:56.726 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-18 13:39:56.726 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['train', 'validation'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-18 13:39:57.918 INFO    absl: Load dataset info from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0
2022-07-18 13:39:57.920 INFO    absl: Reusing dataset patch_camelyon (C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0)
2022-07-18 13:39:57.921 INFO    absl: Constructing tf.data.Dataset patch_camelyon for split ['test'], from C:\Users\lucas\tensorflow_datasets\patch_camelyon\2.0.0


Store features:

In [7]:
write_tfrecord_dataset(dataset=train_ds, model=model, file_path=file_path + '/train/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.30s/it]


In [8]:
write_tfrecord_dataset(dataset=test_ds, model=model, file_path=file_path + '/test/dataset_')

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:23<00:00,  1.16s/it]


In [8]:
write_tfrecord_dataset(dataset=valid_ds, model=model, file_path=file_path + '/validation/dataset_')

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [07:13<00:00,  1.69s/it]


model output size for one image:

In [6]:
image = list(train_ds.take(1))[0][0][0]
print('Single image shape', image.shape)
image = tf.expand_dims(image, axis=0)
print('Image with batch dimension', image.shape)
print('Model output shape', model.predict(image).shape)

Single image shape (224, 224, 3)
Image with batch dimension (1, 224, 224, 3)
Model output shape (1, 7, 7, 1920)
